## Q.11. 平滑化フィルタ

平滑化フィルタ(3x3)を実装せよ。

平滑化フィルタはフィルタ内の画素の平均値を出力するフィルタである。

In [ ]:
import cv2
import numpy as np

img = cv2.imread("imori.jpg").astype(np.float32)

def MeanFilter(image, kernel_size):
    # 画像がカラー画像か確認する。グレースケールの場合は次元を拡張する。
    if len(image.shape) == 3:
        H, W, C = image.shape
    else : # np.expand_dims()を用いると３次元に拡張される
        H, W, C = np.expand_dims(image, axis=-1)

    # パディングの大きさを決定する
    pad = kernel_size // 2
	# 上段下段、左右一列にパディングを入れたとして、その大きさの出力を用意する
    out = np.zeros((H + pad * 2, W + pad * 2, C), dtype=float)
	# パディングで囲われた内側にもとの画像を貼り付ける
    out[pad: pad + H, pad: pad + W] = image.copy().astype(float)

    # 作成したカーネルを用いてフィルタリングを行う
    tmp = out.copy()
    for y in range(H):
        for x in range(W):
            for c in range(C):
                out[pad+y, pad+x, c] = np.mean(tmp[y:y+kernel_size, x:x+kernel_size, c])

    # 値を越えないようにクリップする
    out = np.clip(out, 0, 255)
    out = out[pad: pad + H, pad: pad + W].astype(np.uint8)
    return out

img_ = MeanFilter(img, 3)

# 保存して確認する
cv2.imwrite("training_IMG/training_11.png", img_)

In [ ]:
%reset -f

## Q.12. モーションフィルタ

モーションフィルタ(3x3)を実装せよ。

モーションフィルタとは対角方向の平均値を取るフィルタであり、次式で定義される。

```bash
  1/3  0   0
[  0  1/3  0 ]
   0   0  1/3
```

In [ ]:
import cv2
import numpy as np

img = cv2.imread("imori.jpg").astype(np.float32)

def MotionFilter(image, kernel_size):
    # 画像がカラー画像か確認する。グレースケールの場合は次元を拡張する。
    if len(image.shape) == 3:
        H, W, C = image.shape
    else : # np.expand_dims()を用いると３次元に拡張される
        H, W, C = np.expand_dims(image, axis=-1)

    # パディングの大きさを決定する
    pad = kernel_size // 2
	# 上段下段、左右一列にパディングを入れたとして、その大きさの出力を用意する
    out = np.zeros((H + pad * 2, W + pad * 2, C), dtype=float)
	# パディングで囲われた内側にもとの画像を貼り付ける
    out[pad: pad + H, pad: pad + W] = image.copy().astype(float)

    # 作成したカーネルを用いてフィルタリングを行う
    tmp = out.copy()
    for y in range(H):
        for x in range(W):
            for c in range(C):
                out[pad+y, pad+x, c] = np.mean(np.diag(tmp[y:y+kernel_size, x:x+kernel_size, c]))

    # 値を越えないようにクリップする
    out = np.clip(out, 0, 255)
    out = out[pad: pad + H, pad: pad + W].astype(np.uint8)
    return out

img_ = MotionFilter(img, 3)

# 保存して確認する
cv2.imwrite("training_IMG/training_12.png", img_)

In [ ]:
%reset -f

## Q.13. MAX-MINフィルタ

MAX-MINフィルタ(3x3)を実装せよ。

MAX-MINフィルタとはフィルタ内の画素の最大値と最小値の差を出力するフィルタであり、**エッジ検出**のフィルタの一つである。
エッジ検出とは画像内の線を検出るすることであり、このような画像内の情報を抜き出す操作を**特徴抽出**と呼ぶ。
エッジ検出では多くの場合、グレースケール画像に対してフィルタリングを行う。


In [ ]:
import cv2
import numpy as np

img = cv2.imread("imori.jpg").astype(np.float32)

def MaxMinFilter(image, kernel_size):
    # 画像がカラー画像か確認する。グレースケールの場合は次元を拡張する。
    if len(image.shape) == 3:
        H, W, C = image.shape
    else : # np.expand_dims()を用いると３次元に拡張される
        H, W, C = np.expand_dims(image, axis=-1)

    # パディングの大きさを決定する
    pad = kernel_size // 2
	# 上段下段、左右一列にパディングを入れたとして、その大きさの出力を用意する
    out = np.zeros((H + pad * 2, W + pad * 2, C), dtype=float)
	# パディングで囲われた内側にもとの画像を貼り付ける
    out[pad: pad + H, pad: pad + W] = image.copy().astype(float)

    # 作成したカーネルを用いてフィルタリングを行う
    tmp = out.copy()
    for y in range(H):
        for x in range(W):
            for c in range(C):
                out[pad+y, pad+x, c] = np.max(tmp[y:y+kernel_size, x:x+kernel_size, c]) - np.min(tmp[y:y+kernel_size, x:x+kernel_size, c])

    # 値を越えないようにクリップする
    out = np.clip(out, 0, 255)
    out = out[pad: pad + H, pad: pad + W].astype(np.uint8)
    return out

img_ = MaxMinFilter(img, 3)

# 保存して確認する
cv2.imwrite("training_IMG/training_13.png", img_)

In [ ]:
%reset -f

## Q.14. 微分フィルタ

微分フィルタ(3x3)を実装せよ。

微分フィルタは輝度の急激な変化が起こっている部分のエッジを取り出すフィルタであり、隣り合う画素同士の差を取る。

グレースケールした画像に適用せよ。

```bash
    (a)縦方向         (b)横方向
      0 -1  0            0 0 0
K = [ 0  1  0 ]   K = [ -1 1 0 ]
      0  0  0            0 0 0
```


In [ ]:
import cv2
import numpy as np

img = cv2.imread("imori.jpg").astype(np.float32)

def DifferentialFilter(image, kernel_size):
    # グレースケール化
    img = 0.2126*image[:,:,2] + 0.7152*image[:,:,1] + 0.0722*image[:,:,0]

    # 変数の型を unit8 にする
    img = img.astype(np.uint8)
    H, W = img.shape

    # パディングの大きさを決定する
    pad = kernel_size // 2
	# 上段下段、左右一列にパディングを入れたとして、その大きさの出力を用意する
    out = np.zeros((H+pad*2, W+pad*2), dtype=float)
	# パディングで囲われた内側にもとの画像を貼り付ける
    out[pad:pad+H, pad:pad+W] = img.copy().astype(float)

    # 縦・横それぞれの出力を用意する
    out_v = out.copy()
    out_h = out.copy()

	# 縦方向のカーネル
    Kv = [[0., -1., 0.],[0., 1., 0.],[0., 0., 0.]]
	# 横方向のカーネル
    Kh = [[0., 0., 0.],[-1., 1., 0.], [0., 0., 0.]]

    # 作成したカーネルを用いてフィルタリングを行う
    tmp = out.copy()
    for y in range(H):
        for x in range(W):
                out_v[pad+y, pad+x] = np.sum(Kv*tmp[y:y+kernel_size, x:x+kernel_size])
                out_h[pad+y, pad+x] = np.sum(Kh*tmp[y:y+kernel_size, x:x+kernel_size])

    # 値を越えないようにクリップする
    out_v = np.clip(out_v, 0, 255)
    out_h = np.clip(out_h, 0, 255)
    out_v = out_v[pad:pad+H, pad:pad+W].astype(np.uint8)
    out_h = out_v[pad:pad+H, pad:pad+W].astype(np.uint8)

    return out_v, out_h

img_v, img_h = DifferentialFilter(img, 3)

# 保存して確認する
cv2.imwrite("training_IMG/training_14_v.png", img_v)
cv2.imwrite("training_IMG/training_14_h.png", img_h)

In [ ]:
%reset -f

## Q.15. Sobelフィルタ

Sobelフィルタ(3x3)を実装せよ。

ソーベルフィルタ(Sobelフィルタ)は特定方向（縦や横）のエッジのみを抽出するフィルタであり、次式でそれぞれ定義される。

```bash
    (a)縦方向       (b)横方向
       1  2  1           1  0 -1
K = [  0  0  0 ]   K = [ 2  0 -2 ]
      -1 -2 -1           1  0 -1
```

In [ ]:
import cv2
import numpy as np

img = cv2.imread("imori.jpg").astype(np.float32)

def SobelFilter(image, kernel_size):
    # グレースケール化
    img = 0.2126*image[:,:,2] + 0.7152*image[:,:,1] + 0.0722*image[:,:,0]

    # 変数の型を unit8 にする
    img = img.astype(np.uint8)
    H, W = img.shape

    # パディングの大きさを決定する
    pad = kernel_size // 2
	# 上段下段、左右一列にパディングを入れたとして、その大きさの出力を用意する
    out = np.zeros((H+pad*2, W+pad*2), dtype=float)
	# パディングで囲われた内側にもとの画像を貼り付ける
    out[pad:pad+H, pad:pad+W] = img.copy().astype(float)

    # 縦・横それぞれの出力を用意する
    out_v = out.copy()
    out_h = out.copy()

	# 縦方向のカーネル
    Kv = [[1., 2., 1.],[0., 0., 0.],[-1., -2., -1.]]
	# 横方向のカーネル
    Kh = [[1., 0., -1.],[2., 0., -2.], [1., 0., -1.]]

    # 作成したカーネルを用いてフィルタリングを行う
    tmp = out.copy()
    for y in range(H):
        for x in range(W):
                out_v[pad+y, pad+x] = np.sum(Kv*tmp[y:y+kernel_size, x:x+kernel_size])
                out_h[pad+y, pad+x] = np.sum(Kh*tmp[y:y+kernel_size, x:x+kernel_size])

    # 値を越えないようにクリップする
    out_v = np.clip(out_v, 0, 255)
    out_h = np.clip(out_h, 0, 255)
    out_v = out_v[pad:pad+H, pad:pad+W].astype(np.uint8)
    out_h = out_v[pad:pad+H, pad:pad+W].astype(np.uint8)

    return out_v, out_h

img_v, img_h = SobelFilter(img, 3)

# 保存して確認する
cv2.imwrite("training_IMG/training_15_v.png", img_v)
cv2.imwrite("training_IMG/training_15_h.png", img_h)

In [ ]:
%reset -f

## Q.16. Prewittフィルタ

Prewittフィルタ(3x3)を実装せよ。

Prewittフィルタはエッジ抽出フィルタの一種であり、次式で定義される。

```bash
    (a)縦方向          (b)横方向
      -1 -1 -1          -1 0 1
K = [  0  0  0 ]  K = [ -1 0 1 ]
       1  1  1          -1 0 1
```


In [ ]:
import cv2
import numpy as np

img = cv2.imread("imori.jpg").astype(np.float32)

def PrewittFilter(image, kernel_size):
    # グレースケール化
    img = 0.2126*image[:,:,2] + 0.7152*image[:,:,1] + 0.0722*image[:,:,0]

    # 変数の型を unit8 にする
    img = img.astype(np.uint8)
    H, W = img.shape

    # パディングの大きさを決定する
    pad = kernel_size // 2
	# 上段下段、左右一列にパディングを入れたとして、その大きさの出力を用意する
    out = np.zeros((H+pad*2, W+pad*2), dtype=float)
	# パディングで囲われた内側にもとの画像を貼り付ける
    out[pad:pad+H, pad:pad+W] = img.copy().astype(float)

    # 縦・横それぞれの出力を用意する
    out_v = out.copy()
    out_h = out.copy()

	# 縦方向のカーネル
    Kv = [[-1., -1., -1.],[0., 0., 0.],[1., 1., 1.]]
	# 横方向のカーネル
    Kh = [[-1., 0., 1.],[-1., 0., 1.], [-1., 0., 1.]]

    # 作成したカーネルを用いてフィルタリングを行う
    tmp = out.copy()
    for y in range(H):
        for x in range(W):
                out_v[pad+y, pad+x] = np.sum(Kv*tmp[y:y+kernel_size, x:x+kernel_size])
                out_h[pad+y, pad+x] = np.sum(Kh*tmp[y:y+kernel_size, x:x+kernel_size])

    # 値を越えないようにクリップする
    out_v = np.clip(out_v, 0, 255)
    out_h = np.clip(out_h, 0, 255)
    out_v = out_v[pad:pad+H, pad:pad+W].astype(np.uint8)
    out_h = out_v[pad:pad+H, pad:pad+W].astype(np.uint8)

    return out_v, out_h

img_v, img_h = PrewittFilter(img, 3)

# 保存して確認する
cv2.imwrite("training_IMG/training_16_v.png", img_v)
cv2.imwrite("training_IMG/training_16_h.png", img_h)

In [ ]:
%reset -f

## Q.17. Laplacianフィルタ

Laplacianフィルタを実装せよ。

Laplacian（ラプラシアン）フィルタとは輝度の二次微分をとることでエッジ検出を行うフィルタである。

デジタル画像は離散データであるので、x方向・y方向の一次微分は、それぞれ次式で表される。

```bash
Ix(x,y) = (I(x+1, y) - I(x,y)) / ((x+1)-x) = I(x+1, y) - I(x,y)
Iy(x,y) = (I(x, y+1) - I(x,y)) / ((y+1)-y) = I(x, y+1) - I(x,y)
```

さらに二次微分は、次式で表される。

```bash
Ixx(x,y) = (Ix(x,y) - Ix(x-1,y)) / ((x+1)-x) = Ix(x,y) - Ix(x-1,y)
         = (I(x+1, y) - I(x,y)) - (I(x, y) - I(x-1,y))
         = I(x+1,y) - 2 * I(x,y) + I(x-1,y)
Iyy(x,y) = ... = I(x,y+1) - 2 * I(x,y) + I(x,y-1)
```

これらより、ラプラシアン は次式で定義される。

```bash
D^2 I(x,y) = Ixx(x,y) + Iyy(x,y)
           = I(x-1,y) + I(x,y-1) - 4 * I(x,y) + I(x+1,y) + I(x,y+1)
```

これをカーネル化すると、次のようになる。

```bash
      0  1  0
K = [ 1 -4  1 ]
      0  1  0
```


In [ ]:
import cv2
import numpy as np

img = cv2.imread("imori.jpg").astype(np.float32)

def LaplacianFilter(image, kernel_size):
    # グレースケール化
    img = 0.2126*image[:,:,2] + 0.7152*image[:,:,1] + 0.0722*image[:,:,0]

    # 変数の型を unit8 にする
    img = img.astype(np.uint8)
    H, W = img.shape

    # パディングの大きさを決定する
    pad = kernel_size // 2
	# 上段下段、左右一列にパディングを入れたとして、その大きさの出力を用意する
    out = np.zeros((H+pad*2, W+pad*2), dtype=float)
	# パディングで囲われた内側にもとの画像を貼り付ける
    out[pad:pad+H, pad:pad+W] = img.copy().astype(float)

	# ラプラシアンカーネルの作成
    K = [[0., 1., 0.],[1., -4., 1.],[0., 1., 0.]]

    # 作成したカーネルを用いてフィルタリングを行う
    tmp = out.copy()
    for y in range(H):
        for x in range(W):
                out[pad+y, pad+x] = np.sum(K*tmp[y:y+kernel_size, x:x+kernel_size])

    # 値を越えないようにクリップする
    out = np.clip(out, 0, 255)
    out = out[pad:pad+H, pad:pad+W].astype(np.uint8)

    return out

img_ = LaplacianFilter(img, 3)

# 保存して確認する
cv2.imwrite("training_IMG/training_17.png", img_)

In [ ]:
%reset -f

## Q.18. Embossフィルタ

Embossフィルタを実装せよ。

Embossフィルタとは輪郭部分を浮き出しにするフィルタで、次式で定義される。

```bash
      -2 -1  0
K = [ -1  1  1 ]
       0  1  2
```


In [ ]:
import cv2
import numpy as np

img = cv2.imread("imori.jpg").astype(np.float32)

def EmbossFilter(image, kernel_size):
    # グレースケール化
    img = 0.2126*image[:,:,2] + 0.7152*image[:,:,1] + 0.0722*image[:,:,0]

    # 変数の型を unit8 にする
    img = img.astype(np.uint8)
    H, W = img.shape

    # パディングの大きさを決定する
    pad = kernel_size // 2
	# 上段下段、左右一列にパディングを入れたとして、その大きさの出力を用意する
    out = np.zeros((H+pad*2, W+pad*2), dtype=float)
	# パディングで囲われた内側にもとの画像を貼り付ける
    out[pad:pad+H, pad:pad+W] = img.copy().astype(float)

	# ラプラシアンカーネルの作成
    K = [[-2., -1., 0.],[-1., 1., 1.],[0., 1., 2.]]

    # 作成したカーネルを用いてフィルタリングを行う
    tmp = out.copy()
    for y in range(H):
        for x in range(W):
                out[pad+y, pad+x] = np.sum(K*tmp[y:y+kernel_size, x:x+kernel_size])

    # 値を越えないようにクリップする
    out = np.clip(out, 0, 255)
    out = out[pad:pad+H, pad:pad+W].astype(np.uint8)

    return out

img_ = EmbossFilter(img, 3)

# 保存して確認する
cv2.imwrite("training_IMG/training_18.png", img_)

In [ ]:
%reset -f

## Q.19. LoGフィルタ

LoGフィルタ(sigma=3、カーネルサイズ=5)を実装し、*imori_noise.jpg*のエッジを検出せよ。

LoGフィルタとはLaplacian of Gaussianであり、ガウシアンフィルタで画像を平滑化した後にラプラシアンフィルタで輪郭を取り出すフィルタである。

Laplcianフィルタは二次微分をとるのでノイズが強調されるのを防ぐために、予めGaussianフィルタでノイズを抑える。

LoGフィルタは次式で定義される。

```bash
LoG(x,y) = (x^2 + y^2 - sigma^2) / (2 * pi * sigma^6) * exp(-(x^2+y^2) / (2*sigma^2))
```


In [ ]:
import cv2
import numpy as np

img = cv2.imread("imori_noise.jpg").astype(np.float32)

def LoGFilter(image, sigma, kernel_size):
    # グレースケール化
    img = 0.2126*image[:,:,2] + 0.7152*image[:,:,1] + 0.0722*image[:,:,0]

    # 変数の型を unit8 にする
    img = img.astype(np.uint8)
    H, W = img.shape

    # パディングの大きさを決定する
    pad = kernel_size // 2
	# 上段下段、左右一列にパディングを入れたとして、その大きさの出力を用意する
    out = np.zeros((H+pad*2, W+pad*2), dtype=float)
	# パディングで囲われた内側にもとの画像を貼り付ける
    out[pad:pad+H, pad:pad+W] = img.copy().astype(float)

	# LoG Kernel
    K = np.zeros((kernel_size, kernel_size), dtype=float)
    for x in range(-pad, -pad+kernel_size):
        for y in range(-pad, -pad+kernel_size):
            K[y+pad, x+pad] = (x**2 + y**2 - sigma**2)*np.exp(-(x**2 + y**2)/(2*(sigma**2)))
    K /= (2*np.pi*(sigma**6))
    K /= K.sum()

    # 作成したカーネルを用いてフィルタリングを行う
    tmp = out.copy()
    for y in range(H):
        for x in range(W):
                out[pad+y, pad+x] = np.sum(K*tmp[y:y+kernel_size, x:x+kernel_size])

    # 値を越えないようにクリップする
    out = np.clip(out, 0, 255)
    out = out[pad:pad+H, pad:pad+W].astype(np.uint8)

    return out

img_ = LoGFilter(img, 3, 5)

# 保存して確認する
cv2.imwrite("training_IMG/training_19.png", img_)

In [ ]:
%reset -f

## Q.20. ヒストグラム表示

matplotlibを用いて*imori_dark.jpg*のヒストグラムを表示せよ。

ヒストグラムとは画素の出現回数をグラフにしたものである。
matplotlibではhist()という関数がすでにあるので、それを利用する。


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread("imori_dark.jpg").astype(np.float32)

plt.hist(img.ravel(), bins=255, rwidth=0.8, range=(0, 255))
plt.savefig("training_20.png")
plt.show()
